In [1]:
import time

notebook_start_time = time.time()

In [2]:
import sys
from pathlib import Path

root_dir = str(Path().absolute().parent)
sys.path.append(root_dir)

In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

from pprint import pprint
from loguru import logger

import polars as pl
import torch
from sentence_transformers import SentenceTransformer

from recsys.hopsworks_integration import feature_store
from recsys.config import settings
from recsys.raw_data_sources import load_datasets
from recsys.features.articles import compute_features_articles, generate_embeddings_for_dataframe
from recsys.features.customers import compute_features_customers, DatasetSampler
from recsys.features.transactions import compute_features_transactions
from recsys.features.interaction import generate_interaction_data

## Connect to Hopsworks Feature Store

In [36]:
project, fs = feature_store.get_feature_store()

2025-07-26 19:01:24.701 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:10 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-07-26 19:01:24,702 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-07-26 19:01:24,882 INFO: Initializing external client
2025-07-26 19:01:24,882 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-07-26 19:01:26,109 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/90261


## The H&M dataset

### Articles data

In [5]:
articles_df = load_datasets.extract_articles_df()
articles_df.shape

(105542, 25)

In [6]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"


In [7]:
articles_df.null_count()

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,416


#### feature engineering

In [8]:
articles_df = compute_features_articles(articles_df)
articles_df.shape

(105542, 27)

In [9]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
str,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,u32,str,str
"""108775015""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""https://repo.hops.works/dev/jd…"
"""108775044""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""https://repo.hops.works/dev/jd…"
"""108775051""",108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",13,"""Strap top (1) - Vest top in Ga…","""https://repo.hops.works/dev/jd…"


##### Create embeddings from the articles description

In [10]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2025-07-26 18:28:59.265 | INFO     | __main__:<module>:2 - Item 1:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Dark Black (Black)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-07-26 18:28:59.266 | INFO     | __main__:<module>:2 - Item 2:
Strap top - Vest top in Garment Upper body
Appearance: Solid
Color: Light White (White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.
2025-07-26 18:28:59.266 | INFO     | __main__:<module>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
Appearance: Stripe
Color: Dusty Light White (Off White)
Category: Ladieswear - Womens Everyday Basics - Jersey Basic
Details: Jersey top with narrow shoulder straps.


In [11]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)

2025-07-26 18:28:59.374 | INFO     | __main__:<module>:9 - Loading 'all-MiniLM-L6-v2' embedding model to device='cuda'


In [12]:
# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2025-07-26 18:28:59,416 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [13]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=128
)

Generating embeddings: 100%|██████████| 105542/105542 [01:15<00:00, 1395.71it/s]


In [14]:
articles_df[["article_description", "embeddings"]].head(3)

article_description,embeddings
str,list[f64]
"""Strap top - Vest top in Garmen…","[-0.026782, 0.082344, … 0.022782]"
"""Strap top - Vest top in Garmen…","[-0.010396, 0.089874, … 0.022564]"
"""Strap top (1) - Vest top in Ga…","[-0.032753, 0.091124, … 0.022804]"


In [15]:
from IPython.display import HTML, display

image_urls = articles_df["image_url"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10px; max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))

### Customers Data

In [16]:
customers_df = load_datasets.extract_customers_df()
customers_df.shape

(1371980, 7)

In [17]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [18]:
customers_df.null_count()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
u32,u32,u32,u32,u32,u32,u32
0,895050,907576,6062,16009,15861,0


##### feature engineering

In [19]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

In [20]:
customers_df.head(3)

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""00000dbacae5abe5e23885899a1fa4…","""ACTIVE""",49.0,"""52043ee2162cf5aa7ee79974281641…","""46-55"""
"""0000423b00ade91418cceaf3b26c6a…","""ACTIVE""",25.0,"""2973abc54daa8a5f8ccfe9362140c6…","""19-25"""
"""000058a12d5b43e67d225668fa1f8d…","""ACTIVE""",24.0,"""64f17e6a330a85798e4998f62d0930…","""19-25"""


### Transactions Data

In [21]:
transactions_df =load_datasets.extract_transactions_df()
transactions_df.shape

(31788324, 5)

In [22]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id
date,str,i64,f64,i64
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2


#### feature engineering

In [23]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(31788324, 9)

In [24]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
i64,str,str,f64,i64,i32,i8,i8,i8
0,"""000058a12d5b43e67d225668fa1f8d…","""663713001""",0.050831,2,2018,9,20,4
0,"""000058a12d5b43e67d225668fa1f8d…","""541518023""",0.030492,2,2018,9,20,4
0,"""00007d2de826758b65a93dd24ce629…","""505221004""",0.015237,2,2018,9,20,4


In [25]:
sampler = DatasetSampler(size=settings.CUSTOMER_DATA_SIZE)
dataset_subset = sampler.sample(
    customers_df=customers_df, transations_df=transactions_df
)
customers_df = dataset_subset["customers"]
transactions_df = dataset_subset["transactions"]

2025-07-26 18:31:19.089 | INFO     | recsys.features.customers:sample:77 - Sampling 1000 customers.
2025-07-26 18:31:19.090 | INFO     | recsys.features.customers:sample:80 - Number of transactions for all the customers: 31788324
2025-07-26 18:31:19.276 | INFO     | recsys.features.customers:sample:86 - Number of transactions for the 1000 sampled customers: 23799


In [26]:
transactions_df.shape

(23799, 9)

### Interaction data

In [27]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing customer chunks: 100%|██████████| 1/1 [00:05<00:00,  5.24s/it]


(135770, 5)

In [28]:
interaction_df.head()

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
-370800000,"""00b203a32faa3d007dba198ef27c15…","""673630003""",0,"""START"""
-352800000,"""00b203a32faa3d007dba198ef27c15…","""803647007""",0,"""673630003"""
-349200000,"""00b203a32faa3d007dba198ef27c15…","""850241004""",0,"""803647007"""
-349200000,"""00b203a32faa3d007dba198ef27c15…","""525518005""",0,"""850241004"""
-349200000,"""00b203a32faa3d007dba198ef27c15…","""743630003""",0,"""525518005"""


In [29]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,73695
1,38276
2,23799


Here is what each score means:

    0 : No interaction between a customer and an item
    1 : A customer clicked an item
    2 : A customer bought an item

### Create Hopsworks Feature Groups

In [30]:
logger.info("Uploading 'customers' Feature Group to Hopsworks.")
customers_fg = feature_store.create_customers_feature_group(
    fs, df=customers_df, online_enabled=True
)

logger.info("✅ Uploaded 'customers' Feature Group to Hopsworks!")

2025-07-26 18:31:24.864 | INFO     | __main__:<module>:1 - Uploading 'customers' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 1000/1000 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: customers_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/customers_1_offline_fg_materialization/executions
2025-07-26 18:31:41,943 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-26 18:31:47,405 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-26 18:33:27,054 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-07-26 18:33:29,732 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-26 18:33:29,877 INFO: Waiting for log aggregation to finish.
2025-07-26 18:33:49,845 INFO: Execution finished successfully.


Online data ingestion progress: 0.00% |          | Rows 2000/?
2025-07-26 18:33:52.920 | INFO     | __main__:<module>:6 - ✅ Uploaded 'customers' Feature Group to Hopsworks!


In [ ]:
logger.info("Uploading 'articles' Feature Group to Hopsworks.")
articles_fg = feature_store.create_articles_feature_group(
    fs,
    df=articles_df,
    articles_description_embedding_dim=model.get_sentence_embedding_dimension(),
    online_enabled=True,
)
logger.info("✅ Uploaded 'articles' Feature Group to Hopsworks!")

2025-07-26 18:33:52.966 | INFO     | __main__:<module>:1 - Uploading 'articles' Feature Group to Hopsworks.


Uploading Dataframe: 100.00% |██████████| Rows 105542/105542 | Elapsed Time: 05:08 | Remaining Time: 00:00


Launching job: articles_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/articles_1_offline_fg_materialization/executions
2025-07-26 18:39:14,616 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-26 18:39:17,894 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-26 18:42:28,647 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-26 18:42:28,795 INFO: Waiting for log aggregation to finish.
2025-07-26 18:42:46,126 INFO: Execution finished successfully.


Online data ingestion progress: 7.28% |▋         | Rows 7680/105542

In [34]:
logger.info("Uploading 'transactions' Feature Group to Hopsworks.")
trans_fg = feature_store.create_transactions_feature_group(
    fs=fs, df=transactions_df, online_enabled=True
)
logger.info("✅ Uploaded 'transactions' Feature Group to Hopsworks!")

2025-07-26 18:56:58.114 | INFO     | __main__:<module>:1 - Uploading 'transactions' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 23799/23799 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/transactions_1_offline_fg_materialization/executions
2025-07-26 18:57:13,510 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-26 18:57:19,430 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-26 18:59:07,682 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-26 18:59:07,828 INFO: Waiting for log aggregation to finish.
2025-07-26 18:59:33,964 INFO: Execution finished successfully.


Online data ingestion progress: 100.00% |██████████| Rows 23799/23799
2025-07-26 18:59:40.564 | INFO     | __main__:<module>:5 - ✅ Uploaded 'transactions' Feature Group to Hopsworks!


In [37]:
logger.info("Uploading 'interactions' Feature Group to Hopsworks.")
interactions_fg = feature_store.create_interactions_feature_group(
    fs=fs, df=interaction_df, online_enabled=True
)
logger.info("✅ Uploaded 'interactions' Feature Group to Hopsworks!!")

2025-07-26 19:01:29.693 | INFO     | __main__:<module>:1 - Uploading 'interactions' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 135770/135770 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/90261/jobs/named/interactions_1_offline_fg_materialization/executions
2025-07-26 19:01:54,562 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-07-26 19:01:57,271 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-07-26 19:03:53,546 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-07-26 19:03:53,687 INFO: Waiting for log aggregation to finish.
2025-07-26 19:04:07,375 INFO: Execution finished successfully.


Online data ingestion progress: 100.00% |██████████| Rows 135770/135770
2025-07-26 19:04:10.384 | INFO     | __main__:<module>:5 - ✅ Uploaded 'interactions' Feature Group to Hopsworks!!
